In [1]:
import pandas as pd
import os

In [2]:
table = pd.read_csv(os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\Full_Data_Table.csv')

C:\Users\harri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (5,16,19,23,24,25,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
#Aggregate all relevant forest-based land cover types
def dileneate_forests(x):
    return (x['Deciduous Forest'] + x['Evergreen Forest']+x['Mixed Forest']+x['Woody Wetlands'])

In [5]:
table['Forest_Total'] = table.apply(lambda x: dileneate_forests(x), axis = 1)

In [6]:
def class_forest(x):
    if x['Forest_Total'] > .1:
        return 1
    else:
        return 0

In [7]:
#calculating the total forested parcels owned for each unique owner
table['Forest_Parcel'] = table.apply(lambda x: class_forest(x), axis = 1)
fparc_dict = table.groupby(['Unq_ID']).sum()[['Forest_Parcel']].to_dict()
table['Total_Forest_Parcels_Owned'] = table.Unq_ID.map(fparc_dict['Forest_Parcel'])

In [8]:
#Forest totals are in a percentage of total parcel area. Multiply the two gives area of forest. Divide by 4047 gives area in Acres Units
def forest_area(x):
    return x['Forest_Total']*x['Area']/4047

In [9]:
table['Forest_Area'] = table.apply(lambda x: forest_area(x), axis =1)

In [10]:
#Forest Area per Owner (FApO) is the total forest area owned by all the individual owner's properties
#Convert FApO to dictionary to map the values into "Total Forested Acreas Owned"
FApO = table.groupby(['Unq_ID']).sum()[['Forest_Area']]
dd = FApO.to_dict()

In [11]:
table['Total_Forest_Acres_Owned'] = table.Unq_ID.map(dd['Forest_Area'])

In [12]:
def reclass_own_type(x):
    if (x['Own_Type'] == 42) | (x['Own_Type'] == 43):
        return 'Other Private'
    elif x['Own_Type'] == 0:
        return 'Public'
    else:
        return str(x['Own_Type'])

In [13]:
table['GTR-99_Code'] = table.apply(lambda x: reclass_own_type(x), axis =1)

In [14]:
#Need to creat full table and slim table as noted in the script from Jesse

In [15]:
len(table.loc[pd.isnull(table['FIPS_x']) == False])

422308

In [16]:
table['PARCELID'] = pd.Series(range(len(table))) + ('.'+table['FIPS_x'].astype(str)).astype(float)

In [17]:
state_name = table['State'].unique()[0]
#state_name = 'DE'

In [18]:
table.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'OBJECTID_x', 'PARCELAPN_x', 'FIPS_x', 'PRCLDMPID', 'Area',
       'COUNTY_FIPS', 'COUNTY_NAME', 'geometry', 'Centroid_X', 'Centroid_Y',
       'PARCEL_AREA', 'OBJECTID_y', 'PARCELAPN_y', 'FIPS_y', 'TAXAPN',
       'STHSNUM', 'STDIR', 'STSTNAME', 'STSUFFIX', 'STQUADRANT', 'STUNITPRFX',
       'STUNITNUM', 'STCITY', 'STSTATE', 'STZIP', 'STZIP4', 'XCOORD', 'YCOORD',
       'GEOSOURCE', 'ADDRSCORE', 'OWN1', 'OWN2', 'MCAREOFNAM', 'MHSNUMB',
       'MPREDIR', 'MSTNAME', 'MMODE', 'MQUADRNT', 'MUNITPRFX', 'MUNITNUM',
       'MCITY', 'MSTATE', 'MZIP', 'MZIP4', 'OWNPHONE', 'LOCATIONID',
       'PROPDMPID', 'ASMTVERSID', 'State', 'Unq_ID', 'comb_addr', 'place_id',
       'Owners', 'Simple_Owners', 'initial_class', 'Total_Parcels_Owned',
       'Own_Type', 'JOIN_INDEX', 'Open Water', 'Developed, Open Space',
       'Developed, Low Intensity', 'Developed, Medium Intensity',
       'Developed, High Intensity', 'Barren

In [19]:
table = table.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Area', 
                    'geometry', 'GTR-99_Code', 'FIPS_y', 'OBJECTID_x', 'OBJECTID_y', 'PARCELAPN_y', 'State',
                    'comb_addr', 'place_id', 'Owners','Simple_Owners', 'initial_class'], axis=1)

In [20]:
table = table.rename(columns={'PARCELAPN_x': 'PARCELAPN', 'FIPS_x': 'FIPS', 'Centroid_X': 'CENTROID_LONG_EPSG4269', 
                     'Centroid_Y': 'CENTROID_LAT_EPSG4269', 'Own_Type' : 'OWNCD', 'Unq_ID' : 'OWNERSHIP_ID', 
                     'Open Water' : 'NLCD_11_PROP', 'Developed, Open Space': 'NLCD_21_PROP', 'Developed, Low Intensity': 'NLCD_22_PROP',
                      'Developed, Medium Intensity': 'NLCD_23_PROP', 'Developed, High Intensity': 'NLCD_24_PROP',
                      'Barren Land': 'NLCD_31_PROP', 'Deciduous Forest': 'NLCD_41_PROP', 'Evergreen Forest': 'NLCD_42_PROP',
                      'Mixed Forest': 'NLCD_43_PROP','Shrub/Scrub': 'NLCD_52_PROP', 'Herbaceuous': 'NLCD_71_PROP',
                      'Hay/Pasture': 'NLCD_81_PROP', 'Cultivated Crops': 'NLCD_82_PROP', 'Woody Wetlands': 'NLCD_90_PROP',
                      'Emergent Herbaceuous Wetlands': 'NLCD_95_PROP'})

In [17]:
os.mkdir(os.path.dirname(os.getcwd())+'\\OUTPUTS\\FINAL\\TABLES\\'+state_name)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'D:\\Documents\\OwnershipMap\\New_Script\\Test_ENV\\OUTPUTS\\FINAL\\TABLES\\DE'

In [22]:
table.columns

Index(['PARCELAPN', 'FIPS', 'PRCLDMPID', 'COUNTY_FIPS', 'COUNTY_NAME',
       'CENTROID_LONG_EPSG4269', 'CENTROID_LAT_EPSG4269', 'PARCEL_AREA',
       'TAXAPN', 'STHSNUM', 'STDIR', 'STSTNAME', 'STSUFFIX', 'STQUADRANT',
       'STUNITPRFX', 'STUNITNUM', 'STCITY', 'STSTATE', 'STZIP', 'STZIP4',
       'XCOORD', 'YCOORD', 'GEOSOURCE', 'ADDRSCORE', 'OWN1', 'OWN2',
       'MCAREOFNAM', 'MHSNUMB', 'MPREDIR', 'MSTNAME', 'MMODE', 'MQUADRNT',
       'MUNITPRFX', 'MUNITNUM', 'MCITY', 'MSTATE', 'MZIP', 'MZIP4', 'OWNPHONE',
       'LOCATIONID', 'PROPDMPID', 'ASMTVERSID', 'OWNERSHIP_ID',
       'Total_Parcels_Owned', 'OWNCD', 'JOIN_INDEX', 'NLCD_11_PROP',
       'NLCD_21_PROP', 'NLCD_22_PROP', 'NLCD_23_PROP', 'NLCD_24_PROP',
       'NLCD_31_PROP', 'NLCD_41_PROP', 'NLCD_42_PROP', 'NLCD_43_PROP',
       'NLCD_52_PROP', 'NLCD_71_PROP', 'NLCD_81_PROP', 'NLCD_82_PROP',
       'NLCD_90_PROP', 'NLCD_95_PROP', 'Forest_Total', 'Forest_Parcel',
       'Total_Forest_Parcels_Owned', 'Forest_Area', 'Total_Forest

In [23]:
table.to_csv(os.path.dirname(os.getcwd())+'\\OUTPUTS\\FINAL\\TABLES\\'+state_name+'\\DELAWARE_PARCELS_FULL.csv')

In [24]:
table2 = table[['JOIN_INDEX', 'PARCELID', 'PARCEL_AREA', 'CENTROID_LONG_EPSG4269', 'CENTROID_LAT_EPSG4269',
               'FIPS', 'COUNTY_NAME', 'OWNERSHIP_ID', 'OWNCD', 'NLCD_11_PROP', 'NLCD_21_PROP', 'NLCD_22_PROP',
                'NLCD_23_PROP', 'NLCD_24_PROP', 'NLCD_31_PROP', 'NLCD_41_PROP',
                'NLCD_42_PROP', 'NLCD_43_PROP', 'NLCD_52_PROP', 'NLCD_71_PROP',
                'NLCD_81_PROP', 'NLCD_82_PROP', 'NLCD_90_PROP', 'NLCD_95_PROP', 
               'Total_Parcels_Owned', 'Forest_Parcel', 'Total_Forest_Parcels_Owned', 
               'Forest_Area', 'Total_Forest_Acres_Owned']]

In [25]:
table2.to_csv(os.path.dirname(os.getcwd())+'\\OUTPUTS\\FINAL\\TABLES\\'+state_name+'\\DELAWARE_PARCELS_REDUCED.csv') 

In [26]:
print(state_name, "Full and reduced tables created")

DE Full and reduced tables created
